# Bouncing balls in a 2D box

We're going to make a bunch of balls bounce around in a 2D box.  Sounds like fun?  It is also on the path to being scientifically interesting.  

Instead, thing about simulating and visualizing a gas of non-interacting atoms moving in a 2D box.  What does that mean?

Simulating - using a computer to solve the equations that describe a (simplified) mathematical model of the system of interest.

Visualization - graphically displaying the results so you can see what is going on.

Gas - think of the air around us.  Simplistically, it is comprised of tiny particles (atoms or molecules) in free motion, occasionally bumping into each other or the walls of the box.

Non-interacting - we will ignore the atoms bumping into each other (another turtle code example example illustrates how we can include interactions).  

OK - forget about atoms and molcules - back to balls.

A ball in free motion (i.e., not interacting with anything) just moves in a straight line - it goes, and goes, and goes...  

It looks like this:

In [ ]:
from turtle import *

ball = Turtle()
ball.penup()
ball.shape("circle")
ball.color("blue")
ball.shapesize(1,1,1)

vx, vy = 2, 3    # speed or velocity in the x and y directions
x,y = -300.0,300.0
for i in range(500):
    x += vx
    y -= vy
    ball.setpos(x,y)

A bit boring isn't it.  

Let's put it in a box and make it bounce off the walls.

First, let's draw a square box.

In [ ]:
def drawbox(L):
    ''' Draws and fills a square with corners [L,L] and [-L,-L] '''
    t = Turtle()
    t.color('green','yellow')
    t.penup()
    t.goto((-L,-L))
    t.begin_fill()
    t.pendown()
    for x,y in (-L,L),(L,L),(L,-L),(-L,-L):
        t.goto(x,y)
    t.end_fill()
    t.hideturtle()

L = 300
drawbox(L)

How to bounce the ball off the wall?   

[Aside: We assume no friction, the ball is perfectly elastic, and the wall is perfectly rigid.]

Imagine throwing a ball directly at the wall.  When it bounces  it starts moving in the opposite direction.  
* I.e., its velocity perpendicular to the wall changes sign.  
* If the wall is blocking the $+x$ direction of motion, this means that the $x$-component of the velocity changes sign. 

Our walls are at $-L$ and $+L$ in both the $x$ and $y$ directions.

If the ball has bumped into or overshot a wall we must
* change the sign of the corresponding velocity
* reflect the ball back if it has overshot

The following routine does this

In [ ]:
def bounce(x,vx):
    ''' Bounce the ball off the walls returning new position and velocity '''
    if x >= L:       # Check if ball has hit or gone past the wall
        x = 2*L-x    # If so, move back inside wall by distance we have overshot
        vx = -vx     # and change the sign of the velocity
    elif x <= - L:   # Ditto for wall on the opposite side
        x = -2*L-x
        vx = -vx
    return x,vx      # return the new position and velocity

To bounce the ball, all we need to do is use this new routine each time step to update the $x$ and $y$ coordinate and velocity

In [ ]:
ball.clear()
ball.goto(0,0)
vx, vy = 2, 3    # speed in the x and y directions
x,y = -300.0,300.0
for i in range(500):
    x += vx
    y -= vy
    x, vx = bounce(x, vx)
    y, vy = bounce(y, vy)
    ball.setpos(x,y)

It is convenient to combine the position update (`x += vx`) and the bouncing into one routine.

Also, it is bad practice to refer to global variables, so pass `L` as an argument.

In [ ]:
def update_and_bounce(L,x,vx):
    ''' 
    Advance the position using the velocity and bounce the ball 
    off the walls returning new position and velocity 
    '''
    x += vx
    if x >= L:       # Check if ball has hit or gone past the wall
        x = 2*L-x    # If so, move back inside wall by distance we have overshot
        vx = -vx     # and change the sign of the velocity
    elif x <= - L:   # Ditto for wall on the opposite side
        x = -2*L-x
        vx = -vx
    return x,vx      # return the new position and velocity

Using this new routine the main loop is very simple

In [ ]:
ball.clear()
ball.goto(0,0)
vx, vy = 2, 3    # speed in the x and y directions
x,y = -300.0,300.0
for i in range(500):
    x, vx = update_and_bounce(L, x, vx)
    y, vy = update_and_bounce(L, y, vy)
    ball.setpos(x,y)

How about 2 balls? 

It is now convenient to define a function to make a ball.

We also need separate positions and velocities for each ball.

In [ ]:
def make_ball(color):
    ball = Turtle()
    ball.penup()
    ball.shape("circle")
    ball.color(color)
    ball.shapesize(1,1,1)
    return ball
    
ball = make_ball("blue")
ball2 = make_ball("red")

ball.clear()
ball.goto(0,0)
vx, vy = 2, 3 
x,y = -300.0,300.0

vx2, vy2 = -1,4
x2, y2 = 200,90

x,y = 100.0,-100.0
for i in range(500):
    x, vx = update_and_bounce(L, x, vx)
    y, vy = update_and_bounce(L, y, vy)
    ball.setpos(x,y)
    
    x2, vx2 = update_and_bounce(L, x2, vx2)
    y2, vy2 = update_and_bounce(L, y2, vy2)
    ball2.setpos(x2,y2)

That was not too painful for just 2 balls, but what about 200?  And in a real molecular dynamics simulation we may have millions of atoms.

Instead of using separate variables for each ball, we need to put them in a list.

In [ ]:
from random import uniform, randrange

def rancol():
    ''' returns a random color '''
    colors = ['red','green','blue','orange','purple','pink']
    return colors[randrange(len(colors))]

nballs = 10
balls = [make_ball(rancol()) for i in range(nballs)]
coords = [(uniform(-300,300),uniform(-300,300)) for i in range(nballs)]
velocities=[(uniform(-10,10),uniform(-10,10)) for i in range(nballs)]
for i in range(100):
    for i in range(nballs):
        x,y = coords[i]
        vx,vy = velocities[i]
        x, vx = update_and_bounce(L, x, vx)
        y, vy = update_and_bounce(L, y, vy)
        balls[i].setpos(x,y)
        coords[i] = (x,y)
        velocities[i] = (vx,vy)

Putting it all together into a single complete program

In [ ]:
from turtle import *
from random import uniform, randrange

def drawbox(L):
    ''' Draws and fills a square with corners [L,L] and [-L,-L] '''
    t = Turtle()
    t.color('green','yellow')
    t.penup()
    t.goto((-L,-L))
    t.begin_fill()
    t.pendown()
    for x,y in (-L,L),(L,L),(L,-L),(-L,-L):
        t.goto(x,y)
    t.end_fill()
    t.hideturtle()
    
def update_and_bounce(L,x,vx):
    ''' 
    Advance the position using the velocity and bounce the ball 
    off the walls returning new position and velocity 
    '''
    x += vx
    if x >= L:       # Check if ball has hit or gone past the wall
        x = 2*L-x    # If so, move back inside wall by distance we have overshot
        vx = -vx     # and change the sign of the velocity
    elif x <= - L:   # Ditto for wall on the opposite side
        x = -2*L-x
        vx = -vx
    return x,vx      # return the new position and velocity

def make_ball(color):
    ''' makes a ball of the specified color '''
    ball = Turtle()
    ball.speed(0)    # for fastest animation
    ball.penup()
    ball.shape("circle")
    ball.color(color)
    ball.shapesize(1,1,1)
    return ball

def rancol():
    ''' returns a random color '''
    colors = ['red','green','blue','orange','purple','pink']
    return colors[randrange(len(colors))]

L = 300
nballs = 10

drawbox(L)
balls = [make_ball(rancol()) for i in range(nballs)]
coords = [(uniform(-300,300),uniform(-300,300)) for i in range(nballs)]
velocities=[(uniform(-10,10),uniform(-10,10)) for i in range(nballs)]

for i in range(300):
    for i in range(nballs):
        x,y = coords[i]
        vx,vy = velocities[i]
        x, vx = update_and_bounce(L, x, vx)
        y, vy = update_and_bounce(L, y, vy)
        balls[i].setpos(x,y)
        coords[i] = (x,y)
        velocities[i] = (vx,vy)